<a href="https://colab.research.google.com/github/ArghyaPal/Lewis_Signalling_RVED/blob/master/Lewis_Signaling_RVED.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch.optim as optim
from torchvision import datasets,transforms
import torch.utils
from draw_model import DrawModel
from config import *
from utility import Variable,save_image,xrecons_grid
import torch.nn.utils
import matplotlib.pyplot as plt

In [ ]:
torch.set_default_tensor_type('torch.FloatTensor')

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data/', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor()])),
    batch_size=batch_size, shuffle=False)

In [ ]:
model = DrawModel(T,A,B,z_size,N,dec_size,enc_size, train_loader)
optimizer = optim.Adam(model.parameters(),lr=learning_rate,betas=(beta1,0.999))

In [ ]:
if USE_CUDA:
    model.cuda()

In [ ]:
def train():
    avg_loss = 0
    count = 0
    for epoch in range(epoch_num):
        for data,_ in train_loader:
            bs = data.size()[0]
            data = Variable(data).view(bs, -1)
            optimizer.zero_grad()
            loss, imgs_rec = model.loss(data)
            avg_loss += loss.cpu().data.numpy()
            loss.backward()
            torch.nn.utils.clip_grad_norm(model.parameters(), clip)
            optimizer.step()
            count += 1
            if count % 100 == 0:
                print('Epoch-{}; Count-{}; loss: {};'.format(epoch, count, avg_loss / 100))
                if count % 1000 == 0:
                    torch.save(model.state_dict(),'save/weights_%d.tar'%(count))
                    print("I am here..!")
                    generate_image(imgs_rec, count)
                avg_loss = 0
    torch.save(model.state_dict(), 'save/weights_final.tar')
    generate_image(imgs_rec, count)

In [ ]:
def generate_image(imgs_rec, count):
    reconstructed_image_ = imgs_rec
    generated_image_ = model.generate(batch_size)
    save_image(reconstructed_image_, generated_image_, count)

def save_example_image():
    train_iter = iter(train_loader)
    data, _ = train_iter.next()
    img = data.cpu().numpy().reshape(batch_size, 28, 28)
    imgs = xrecons_grid(img, B, A)
    plt.matshow(imgs, cmap=plt.cm.gray)
    plt.savefig('image/example.png')

In [ ]:
if __name__ == '__main__':
    save_example_image()
    train()